In [40]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.model_selection import train_test_split
from spectral.io import envi
from sklearn import tree


In [6]:

data = np.load('../train_no_clouds.npz')
refls = data['output_rfl']
wv = data['output_wv']
fids = data['output_idx']
#fid_number = data['output_fid']

values_idx, counts_idx = np.unique(fids, return_counts=True)
#indexes = np.unique(fid_number, return_index=True)[1]
#fids = np.array([fid_number[index] for index in sorted(indexes)])

scenes = np.split(refls, 743)
wv_by_scene = np.split(wv, 743)
scenes = np.array(scenes)
wv_by_scene = np.array(wv_by_scene)

scenes = scenes[:,:,60:100]


bad = []
for i in range(wv_by_scene.shape[0]):
    scene = wv_by_scene[i].copy()
    if scene[np.where((scene > 6) | (scene < 0))].shape[0] != 0:
        bad.append(i)

wv_by_scene = np.delete(wv_by_scene, bad, axis = 0)
scenes = np.delete(scenes, bad, axis = 0)
print(scenes.shape)
print(wv_by_scene.shape)

(718, 1242, 40)
(718, 1242)


In [68]:
rad_header = envi.open('../EMIT-scenes/west_africa_scene/EMIT_L1B_RAD_001_20230321T150833_2308010_003_radiance.hdr')
rad_header = envi.open('../EMIT-scenes/west_africa_scene/EMIT_L1B_RAD_001_20230321T150833_2308010_003_radiance.hdr')
wl = rad_header.metadata['wavelength']
band_number = np.arange(60,100)
feature_names = [f'Band {i+1}: {j} nm' for i,j in zip(band_number, wl)]

X_train, X_test, y_train, y_test = train_test_split(scenes, wv_by_scene, test_size = 0.2, random_state = 50)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X_train = X_train.reshape((X_train.shape[0]*X_train.shape[1], X_train.shape[2]))
X_test = X_test.reshape((X_test.shape[0]*X_test.shape[1], X_test.shape[2]))
y_train = y_train.flatten()
y_test = y_test.flatten()
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

C:\Users\vpatro\AppData\Local\anaconda3\envs\isofit_env\lib\site-packages\spectral\io\envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


(574, 1242, 40) (144, 1242, 40) (574, 1242) (144, 1242)
(712908, 40) (178848, 40) (712908,) (178848,)


In [130]:
wl = rad_header.metadata['wavelength']
band_number = np.arange(285)
feature_names = [f'Band {i+1}: {j} nm' for i,j in zip(band_number, wl)]

In [133]:
wl = np.loadtxt('../wavelength.txt').tolist()
wl[59]

820.2846

In [105]:
wl = rad_header.metadata['wavelength'][60:100]
band_number = np.arange(60,100)
feature_names = [f'Band {i}: {j} nm' for i,j in zip(band_number, wl)]

In [ ]:
# from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text


dt = DecisionTreeRegressor(random_state = 0)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)




In [114]:
text_representation = tree.export_text(dt, feature_names = feature_names)
print(type(text_representation))


<class 'str'>


str

In [60]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != tree_.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = "if "
        
        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        rule += " then "
        if class_names is None:
            rule += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            rule += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
        rule += f" | based on {path[-1][1]:,} samples"
        rules += [rule]
        
    return rules

In [116]:
most_features_frame = pd.DataFrame(
    data = dt.feature_importances_,
    columns= ['importance'],
    index = np.arange(40),
).sort_values(by=["importance"], ascending=False)

# print(most_features_frame)
feature_order = most_features_frame.index
top_feature_list = [i for i in feature_order]
type(np.array(feature_names)[np.array(top_feature_list)].tolist())

list

In [125]:
features = np.array(feature_names)[np.array(top_feature_list)]
importances = -np.sort(-dt.feature_importances_)

dat = np.array([features, importances])

dat = dat.T

np.savetxt('../test.txt', dat, fmt="%s", delimiter = ', Importance: ')

In [77]:
most_features_frame.index[:5]

Index([39, 6, 16, 15, 30], dtype='int32')

In [75]:
top_5_feature

Index([39, 6, 16, 15, 30], dtype='int32')

In [78]:
top_10_features = most_features_frame.index[:10]
feature_list = [i for i in top_10_features]
print(np.array(feature_names)[np.array(feature_list)].tolist())

['Band 100: 671.09753 nm', 'Band 67: 425.47214 nm', 'Band 77: 499.77142 nm', 'Band 76: 492.33292 nm', 'Band 91: 604.0098 nm', 'Band 92: 611.4622 nm', 'Band 78: 507.2099 nm', 'Band 66: 418.0536 nm', 'Band 90: 596.55835 nm', 'Band 99: 663.6411 nm']


In [79]:
top_5_feature = most_features_frame.index[:5]
top_5_feature_list = [i for i in top_5_feature]
print(np.array(feature_names)[np.array(top_5_feature_list)].tolist())

['Band 100: 671.09753 nm', 'Band 67: 425.47214 nm', 'Band 77: 499.77142 nm', 'Band 76: 492.33292 nm', 'Band 91: 604.0098 nm']


In [88]:
top_10_feat_vals = most_features_frame['importance'][:10]
print(top_10_feat_vals.tolist())

[0.410319650175523, 0.11923490920742193, 0.0759141154514439, 0.06482371185012689, 0.05623415760519173, 0.05276441400883038, 0.01923693208940236, 0.017782301116688937, 0.014948937679537993, 0.014038495502869335]


In [85]:
most_features_frame = pd.DataFrame(
    data = dt.feature_importances_,
    columns= ['importance'],
    index = np.arange(40),
).sort_values(by=["importance"], ascending=False)

# print(most_features_frame)
top_5_feature = most_features_frame.index[:5]
top_5_feature_list = [i for i in top_5_feature]
print(np.array(feature_names)[np.array(top_5_feature_list)].tolist())

,importance
39,0.410320
6,0.119235
16,0.075914
15,0.064824
30,0.056234
31,0.052764
17,0.019237
5,0.017782
29,0.014949
38,0.014038
